<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/B%C4%B0ST_30_HARMON%C4%B0C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# Fibonacci oranları
FIB_0_618 = 0.618
FIB_1_618 = 1.618
FIB_0_786 = 0.786
FIB_1_272 = 1.272
FIB_2_618 = 2.618
FIB_3_618 = 3.618
FIB_0_382 = 0.382
FIB_0_5 = 0.5
FIB_0_886 = 0.886

# BIST 30 hisseleri (ISIN kodları ve sembolleri)
BIST_30_HISSELERI = {
    "AEFES": "AEFES",  # Anadolu Efes
    "AKBNK": "AKBNK",  # Akbank
    "ARCLK": "ARCLK",  # Arçelik
    "ASELS": "ASELS",  # Aselsan
    "BIMAS": "BIMAS",  # BIM
    "EKGYO": "EKGYO",  # Emlak Konut GYO
    "EREGL": "EREGL",  # Ereğli Demir Çelik
    "FROTO": "FROTO",  # Ford Otosan
    "GARAN": "GARAN",  # Garanti Bankası
    "HALKB": "HALKB",  # Halkbank
    "ISCTR": "ISCTR",  # İş Bankası (C)
    "KCHOL": "KCHOL",  # Koç Holding
    "KOZAA": "KOZAA",  # Koza Altın
    "KOZAL": "KOZAL",  # Koza Madencilik
    "PGSUS": "PGSUS",  # Pegasus
    "SAHOL": "SAHOL",  # Sabancı Holding
    "SASA": "SASA",    # Sasa Polyester
    "SISE": "SISE",    # Şişe Cam
    "TAVHL": "TAVHL",  # TAV Havalimanları
    "TCELL": "TCELL",  # Turkcell
    "THYAO": "THYAO",  # Türk Hava Yolları
    "TKFEN": "TKFEN",  # Tekfen Holding
    "TOASO": "TOASO",  # Tofaş
    "TTKOM": "TTKOM",  # Türk Telekom
    "TUPRS": "TUPRS",  # Tüpraş
    "VAKBN": "VAKBN",  # Vakıfbank
    "YKBNK": "YKBNK",  # Yapı Kredi Bankası
}

# Web scraping ile hisse verilerini çekme
def get_stock_data(hisse):
    url = f"https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/default.aspx?hisse={hisse}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Hisse fiyatını çekme (örnek bir selector, gerçekte sayfa yapısına göre değişir)
    try:
        price = soup.find("span", {"class": "value"}).text.strip()
        price = float(price.replace(",", "."))
        return price
    except Exception as e:
        print(f"{hisse} hissesi için fiyat bulunamadı: {e}")
        return None

# Patern tespit fonksiyonları
def detect_gartley(price_A, price_B, price_C, price_D):
    try:
        AB = abs(price_B - price_A)
        BC = abs(price_C - price_B)
        CD = abs(price_D - price_C)

        AB_CD = CD / AB if AB != 0 else 0
        BC_AB = BC / AB if AB != 0 else 0

        return (FIB_1_272 <= AB_CD <= FIB_1_618) and (FIB_0_618 <= BC_AB <= FIB_0_786)
    except Exception as e:
        print(f"Gartley patern tespitinde hata: {e}")
        return False

def detect_butterfly(price_A, price_B, price_C, price_D):
    try:
        AB = abs(price_B - price_A)
        BC = abs(price_C - price_B)
        CD = abs(price_D - price_C)

        AB_CD = CD / AB if AB != 0 else 0
        BC_AB = BC / AB if AB != 0 else 0

        return (FIB_1_618 <= AB_CD <= FIB_2_618) and (FIB_0_786 <= BC_AB <= FIB_0_886)
    except Exception as e:
        print(f"Kelebek patern tespitinde hata: {e}")
        return False

def detect_bat(price_A, price_B, price_C, price_D):
    try:
        AB = abs(price_B - price_A)
        BC = abs(price_C - price_B)
        CD = abs(price_D - price_C)

        AB_CD = CD / AB if AB != 0 else 0
        BC_AB = BC / AB if AB != 0 else 0

        return (FIB_1_618 <= AB_CD <= FIB_2_618) and (FIB_0_382 <= BC_AB <= FIB_0_5)
    except Exception as e:
        print(f"Yarasa patern tespitinde hata: {e}")
        return False

def detect_crab(price_A, price_B, price_C, price_D):
    try:
        AB = abs(price_B - price_A)
        BC = abs(price_C - price_B)
        CD = abs(price_D - price_C)

        AB_CD = CD / AB if AB != 0 else 0
        BC_AB = BC / AB if AB != 0 else 0

        return (FIB_2_618 <= AB_CD <= FIB_3_618) and (FIB_0_382 <= BC_AB <= FIB_0_618)
    except Exception as e:
        print(f"Çiftleme patern tespitinde hata: {e}")
        return False

# Alarm fonksiyonu
def trigger_alarm(hisse, pattern_name, date):
    print(f"Alarm: {hisse} hissesinde {pattern_name} patern tespit edildi! Tarih: {date}")

# Hisse verilerini çek ve paternleri tara
def tarama_yap():
    # Veri çekme aralığı (son 30 gün)
    end_date = datetime.today()
    start_date = end_date - timedelta(days=30)

    for hisse, ticker in BIST_30_HISSELERI.items():
        try:
            print(f"{hisse} hissesi taranıyor...")
            # Hisse verilerini çek
            prices = []
            for single_date in pd.date_range(start_date, end_date):
                price = get_stock_data(ticker)
                if price is not None:
                    prices.append(price)

            if len(prices) < 4:
                print(f"{hisse} hissesi için yeterli veri bulunamadı.")
                continue

            # Fiyat verileri üzerinde döngü
            for i in range(4, len(prices)):
                price_A = prices[i-4]
                price_B = prices[i-3]
                price_C = prices[i-2]
                price_D = prices[i-1]
                date = (start_date + timedelta(days=i-1)).strftime('%Y-%m-%d')

                # Patern tespiti
                if detect_gartley(price_A, price_B, price_C, price_D):
                    trigger_alarm(hisse, "Gartley", date)
                if detect_butterfly(price_A, price_B, price_C, price_D):
                    trigger_alarm(hisse, "Kelebek", date)
                if detect_bat(price_A, price_B, price_C, price_D):
                    trigger_alarm(hisse, "Yarasa", date)
                if detect_crab(price_A, price_B, price_C, price_D):
                    trigger_alarm(hisse, "Çiftleme", date)

        except Exception as e:
            print(f"{hisse} hissesi için hata oluştu: {e}")

# Tarama işlemini başlat
if __name__ == "__main__":
    print("BIST 30 hisseleri taranıyor...")
    tarama_yap()
    print("Tarama tamamlandı.")

BIST 30 hisseleri taranıyor...
AEFES hissesi taranıyor...
AKBNK hissesi taranıyor...
ARCLK hissesi taranıyor...
ASELS hissesi taranıyor...
BIMAS hissesi taranıyor...
EKGYO hissesi taranıyor...
EREGL hissesi taranıyor...
FROTO hissesi taranıyor...
GARAN hissesi taranıyor...
HALKB hissesi taranıyor...
ISCTR hissesi taranıyor...
KCHOL hissesi taranıyor...
KOZAA hissesi taranıyor...
KOZAL hissesi taranıyor...
PGSUS hissesi taranıyor...
SAHOL hissesi taranıyor...
SASA hissesi taranıyor...
SISE hissesi taranıyor...
TAVHL hissesi taranıyor...
TCELL hissesi taranıyor...
THYAO hissesi taranıyor...
TKFEN hissesi taranıyor...
TOASO hissesi taranıyor...
TTKOM hissesi taranıyor...
TUPRS hissesi taranıyor...
VAKBN hissesi taranıyor...
YKBNK hissesi taranıyor...
Tarama tamamlandı.
